::: column-margin
## Reproducibility
To reproduce all the results, feel free to use this [Colab Notebook](https://colab.research.google.com). But, be aware that in order to run the part of the code that gathers the activations, you will need around 24-25GB of RAM in CPU or close to that VRAM if on GPU. The colab free tier does not provide this amount of resources. You can still run the analysis part of the code with this [dataset](https://huggingface.co/datasets/diegomezp/gemmascope_bilingual_activations). The latter will run on almost any relatively modern computer.
:::

## Introduction

Sparse autoencoders (SAEs) trained on the attention heads and residual streams of large language models have shown great promise at producing seemingly interpretable features [@cunningham2023sparse]. Features gathered from SAEs can be used to understand the inner workings of large language models and even to steer their behaviour in a desired direction [@templeton2024scaling]. 

Is not uncommon to find that some of the features learned by SAEs are multilingual, this is particularly interesting because it suggests that the model has learned to represent and reason through concepts in an abstract way that is independent of the language it is written in. The multilinguality of features, can be viewed as evidence for the *universality of features hypothesis*, which states that learned representations are universal and can form across models and tasks. This is one of the main speculative claims of the mechanistic interpretability agenda [@olah2020zoom].

But, how can we find these multilingual features in a SAE?

Much of the recent work regarding SAEs and mechanistic interpretability, has been about either scaling up the models to make them more powerful [@templeton2024scaling] [@gao2024scaling], finding techniques to make the models better at reconstructing the input [@rajamanoharan2024jumping], or using the learned features to find interesting circuits in the model [@wang2022interpretability]. Many of such endevours end up always finding *some* multilingual features, but they are not the main focus of the work, nor are they systematically searched for.



In this work, we present a systematic, data-driven process to generate a list of candidate bilingual features from a GemmaScope SAE. We define a bilingual interpretability score for each feature, which is dependent on a dataset of equivalent English-Spanish sentences. We then rank the features based on this score and analyze them. Finally, we discuss the potential for extending this methodology to include more that 2 languages.


In [ ]:
#| code-fold: true
#| column: page

from dotenv import load_dotenv

load_dotenv(override=True)


<!-- ## Hopes for Mechanistic Interpretability -->

## Preliminaries: Quick Background on Sparse autoencoders

### Basic Theory

### JumpRelu Sparse autoencoder

### GemmaScope SAEs

## Methodology

<!-- In this section, we will describe the methodology used to find bilingual features in a given SAE. First, we will describe the data collection process, then we will explain how we extract features from the SAE, and finally, we will describe the bilingual interpretability score we use to rank the features. -->

**The driving intuition behind our methodology** is that, inspite of changes in tokenization, word order, general linguistic structure, and even the distribution of feature logits across languages, for a feature to be bilingual, it is necessary that it circumvents these differences and be activated by the same or similar sentences in both languages.

Such condition may not be sufficient, but as we will see, it is a good starting point to find bilingual features in a SAE.

In this section, we will describe the specific methodology that arises from this intuition, which consists of three main steps:

1. **Data Collection**: We gather a dataset of equivalent English-Spanish sentences.
2. **Feature Extraction**: We extract the features from the SAE for each sentence in the dataset.
3. **Bilingual Interpretability Score**: We define a score that measures how similar the activations of a feature are across languages.

### Basic Setup

For our experiments, we used ***Gemma 2-2B*** as our language model on its base pretrained version without any intruction tuning. We focused our experiments on a single SAE from the GemmaScope collection of open-source SAEs [@lieberum2024gemma], specifically, the one with id *gemma-scope-2b-pt-res-canonical/layer_20/width_16k/canonical*. This SAE has 16k features and is trained on the residual stream of the 20th layer of the model. It is the smallest version of this particular hook point, and the choice for its size was made purely for computational reasons.

The 2B version of the Gemma 2 models has 26 layers [@team2024gemma], so a SAE trained on the 20th residual stream is expected to have learned more abstract features than earlier layers. Moreover, we decided to use the residual stream instead of the attention heads because it is an information bottleneck where not only the prior attention head writes to, but also all the later ones, so one should expect that the features learned in this specific point are more abstract and general than those inside the attention mechanism [@elhage2021mathematical].

For our bilingual dataset, we used a small sample of the OPUS Books dataset [@tiedemann-2012-parallel], with equivalent English-Spanish sentences. 


### Feature Extraction

To extract the features from the SAE, we used the `HookedSAETransformer` class from the `SAELens` library. This class allows us to *hook* our SAE to a given language model, and cache the activations of the SAE for a given set of inputs. 

We ran the `HookedSAETransformer` on the English and Spanish versions of the sentences in our dataset, and stored the activations using the `datasets` library from Hugging Face. This data is publicly available at hugging face datasets hub under the name *diegomezp/gemmascope_bilingual_activations*. It contains the activations of the SAE for each sentence in the dataset, in both languages as well as the token ids of the sentences.

We can use such data to visualize some basic activation statistics across dataset. 


### Billingual Interpretability Score

### Results

## Discussion

### Limitations

### Future Work

## Conclusion

## Apendix

### On SAELens and TransformersLens

### Interpretation of Feature Dashboards


In [ ]:
#| code-fold: true
#| column: page

from IPython.display import IFrame

# get a random feature from the SAE
feature_idx = 2647

html_template = "https://neuronpedia.org/{}/{}/{}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"

def get_dashboard_html(sae_release = "gpt2-small", sae_id="7-res-jb", feature_idx=0):
    return html_template.format(sae_release, sae_id, feature_idx)

# Neuronpedia names are usually different to the ones in HF/SAE_Lens
neuronpedia_sae, neuronpedia_id = "gemma-2-2b/20-gemmascope-res-16k".split("/")
html = get_dashboard_html(sae_release = neuronpedia_sae, sae_id=neuronpedia_id, feature_idx=feature_idx)
IFrame(html, width=1200, height=600)

...